In [3]:
#Importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from wordcloud import WordCloud, STOPWORDS
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#Loading data
data = pd.read_csv("chatgpt-reddit-comments.csv", index_col='Unnamed: 0')
data.head()

: 

In [ ]:
#Checking data shape
data.shape

: 

In [ ]:
data.info()

: 

In [ ]:
data.isnull().sum()

: 

In [ ]:
data.dropna(inplace = True)
data.isna().sum()

: 

In [ ]:
subreddit_count = data['subreddit'].value_counts()
subreddit_count

: 

In [ ]:
parent_comment = data['comment_parent_id'].value_counts()
parent_comment

: 

In [ ]:
subreddit_count.plot(kind='bar')
plt.xlabel("Subreddit Name")
plt.ylabel("Comment Count")
plt.title("Comment Frequency by Subreddits")

: 

In [ ]:
data.comment_body[0]

: 

In [ ]:
cloud = WordCloud(stopwords=STOPWORDS, background_color='black', max_words=200).generate(str(data.comment_body))
plt.figure(figsize=(15,10))
plt.clf
plt.imshow(cloud)
plt.axis('off')
plt.show()

: 

In [ ]:
sid = SentimentIntensityAnalyzer()

: 

In [ ]:
data['scores'] = data['comment_body'].apply(lambda comment_body: sid.polarity_scores(comment_body))
data.head()

: 

In [ ]:
data['compound']  = data['scores'].apply(lambda score_dict: score_dict['compound'])
data.head()

: 

In [ ]:
# Function to label the sentiment according to score
def get_tag(val):
    if val <= -0.50:
        return 'Negative'
    elif val >= 0.5:
        return 'Positive'
    else:
        return 'Neutral'

: 

In [ ]:
data['Sentiment'] = data['compound'].apply(get_tag)
data.head()

: 

In [ ]:
data.drop(['scores', 'compound'], axis=1, inplace=True)

: 

In [ ]:
data.head(10)

: 

In [ ]:
sentiment = data['Sentiment'].value_counts()
sentiment

: 

In [ ]:
sentiment.plot(kind='bar')
plt.xlabel("Sentiment")
plt.ylabel("Sentiment Type Count")
plt.title("Sentment Frequenecy")

: 

In [ ]:
data.to_csv('Labeled Data.csv')

: 

In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

: 

In [ ]:
model.summary()

: 